In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = OpenAI()

chat = ChatOpenAI(temperature=0.1)

In [4]:
# a = llm.predict("How many planets are there?")
# b = chat.predict("How many planets are there?")

a, b

('\n\nThere are eight known planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [2]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [6]:


messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian"),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also what is your name?"
    ),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 17.000 chilometri.')

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [4]:
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

In [5]:
chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [9]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}."),
    ]
)

prompt = template.format_messages(
    language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand"
)

In [10]:
chat.predict_messages(prompt)

AIMessage(content='Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [12]:
from langchain.schema import BaseOutputParser

In [13]:
class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        return text.strip().split(",")
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', ' how', ' are', ' you']

In [14]:
class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [19]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do not reply with anything else",
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(max_items=10, question="What are the colors?")

chat.predict_messages(prompt)

AIMessage(content='red, orange, yellow, green, blue, indigo, violet, black, white, gray')

In [21]:
prompt = template.format_messages(max_items=10, question="What are the planets?")

result = chat.predict_messages(prompt)

In [22]:
p = CommaOutputParser()

p.parse(result.content)

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto']

In [1]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(temperature=0.1)

In [2]:
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do not reply with anything else",
        ),
        ("human", "{question}"),
    ]
)

In [3]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [8]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow redipies for any type of cuisine with easy to find ingredient."),
    ("human", "I want to cook {cusine} food"),
])

chef_chain = chef_prompt | chat 

In [9]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipie. If there is no alternative for a food just say ypu don't know how to replace it",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

In [10]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cusine": "indian"
})

AIMessage(content="To make a vegetarian version of Butter Chicken, you can replace the chicken with a plant-based protein alternative such as tofu or paneer. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter or vegetable oil\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream or coconut cream\n- Salt, to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter or vegetable oil in a large pan over medium heat. Add the chopped onions and sauté until they turn golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [2]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow redipies for any type of cuisine with easy to find ingredient."),
    ("human", "I want to cook {cusine} food"),
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipie. If there is no alternative for a food just say ypu don't know how to replace it",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

In [3]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cusine": "indian"
})

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's an easy recipe for you:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.
4. Add the chicken pieces to the pan a

AIMessageChunk(content="To make a vegetarian version of Butter Chicken, you can replace the chicken with a plant-based protein such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream (or coconut cream for a vegan version)\n- Salt, to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, an